Many people have a Network Attached Storage (NAS) server in their home networks to share files and perhaps function as a private cloud. These NAS servers usually offer different ways (protocols) to share files with devices on the network. Most common protocols are:
* [Server Message Block (SMB)](https://en.wikipedia.org/wiki/Server_Message_Block)
  
* [Network File System (NFS)](https://en.wikipedia.org/wiki/Network_File_System)

* [Apple Filing Protocol (AFP)](https://en.wikipedia.org/wiki/Apple_Filing_Protocol)

The SMB protocol is the standard in Windows networks, whereas NFS is used more in Unix/Linux environments. I have a [Synology](https://www.synology.com/) NAS and have both Linux and Microsoft Windows clients connecting to the fileserver. 

SMB works fine on Linux too, but there are a number of advantages in using NFS which I will not detail here. However, certain steps in NFS configuration are especially important to make file permissions work properly/securely with version 4 of the NFS protocol. This has to do with how user accounts on the NAS are mapped and authorized to the user accounts on the Linux client. A quick and dirty setup on the Synology NAS is possible: mapping all connecting NFS clients to a guest account on the NAS. As a consequence, owners of the files created on the NFS share are mapped to user `nobody`. With this setup I ran into issues with using `git` on the NFS fileshares since my Linux user was not the owner of the repository files.

Below I describe how I configured NFSv4 + Kerberos on the Synology NAS and my Linux client in order to work properly with regards to file ownership and permissions. This is by no means an exhaustive description and assumes some prior knowledge on the subjects, but hopefully it gives some form of guidance.

System environment:
- **NFS Server**: Synology DS216+ NAS running DSM 7.1.1-42962 Update 6  
  - NFSv4
  - Kerberos 5 v1.18.3
 
* **NFS Client**: Manjaro Linux running a 6.3 kernel
    * Mounting NFSv4 share on NAS.
    * Acting as Kerberos KDC (Kerberos Distribution Center). Note: in larger networks the KDC would typically be on a separate server. 
    * Kerberos 5 v1.19.4   
  
- **No DNS on local network**


#### Kerberos
NSFv4 clients are authenticated on user level. This can be done using standard UNIX authentication (AUTH_SYS), but then user accounts on client and NAS need to match on UID/GID level as I understand it. The other authentication method uses [Kerberos](https://en.wikipedia.org/wiki/Kerberos_(protocol)) Understanding how Kerberos works (high level) takes some time. It is typically one of those things where you will need to read documentation, try things out, re-read documentation and try some more.

Here are a number of resources which I found useful:
- [Tutorial (kerberos.org)](https://kerberos.org/software/tutorial.html)   
  I suggest to start here.
  
- [Official Documentation (web.mit.edu)](https://web.mit.edu/kerberos/krb5-latest/doc/)   
  This is a collection of separate HTML pages. I found the single pdf file contained in the [releases](http://web.mit.edu/kerberos/dist/) easier to navigate.
  
- [Kerberos on Arch Wiki](https://wiki.archlinux.org/title/Kerberos)   
  I run the Manjaro Distribution, which is based on Arch Linux.
  
- [Kerberos Tutorial Debian/Ubuntu](http://techpubs.spinlocksolutions.com/dklar/kerberos.html)   
  Not all parts are applicable to Manjaro (and perhaps not anymore to Debian either?), but still very informative.

- Linux man files on the various config files, programs and protocols.
  
#### NFS Client
* Enable NFS client services: `sudo systemctl enable nfs-client.target`  
  The [Synology documentation](https://kb.synology.com/en-br/DSM/help/DSM/AdminCenter/file_winmacnfs_nfs?version=7) mentions that _idmapd_ needs to be enabled. However, I noticed that the idmapd manual in Linux says that only NFSv4 servers need to have that daemon running. The client on newer Linux kernels now uses the _nfsidmap_ program to do name mapping. No need to enable idmapd.
* Configure`/etc/hosts`
  The hostname of the client (and KDC) must be mapped to the LAN ip address, _not_ the 127.* address.  
  The hostname of the service/server (NAS) must be mapped to the LAN ip address (unless you have DNS running).

* Kerberos configuration in `/etc/krb5.conf`  
    - Section `[libdefaults]`: set your `default_realm` to a name you want to use for your Kerberos realm. Convention is to use the DNS domain in capital letters, but if you do not use DNS in your local network you can of course pick something else.
    - Section `[realms]`: map the `admin_server` and `kdc`for your realm to the ip address/FQDN of your linux client.
    - Section `[domain_realm]`: map the domain to your real

- Add an administrator user to the Kerberos Access Control List `/var/lib/krb5kdc/kadm5.acl`

* Kerberos database configuration  
  - After initial creation of the database, use `$sudo kadmin_local` to add the administrator principal to the database. After having added this principal, you can from now on use `$kadmin` which interacts with the database _via the kadmind daemon_.
  - Add a standard principal that is your user on the Linux client.
  - Add a service principal for the NAS/NSFv4 server.
 
- Export the principal key for the server to the `/etc/krb5.keytab` file using `$kadmin`.
  Every time you generate a new entry for a principal to the KDC database, a key-version-number (kvno) is incremented. 

* Prepare a mount point for the NFSv4 share.  
  Add an entry in the `/etc/fstab` file (type nfs). It is not necessary to indicate any NFS version or other options, defaults will do.
    
#### NFS Server
* Enable the NFSv4 service in the NAS.
  
     <img src="images/DSM_Files_Services_NFS.png" alt="Image: DSM Control Panel - File Sharing" width="800"/>
     
* Enter NFSv4 domain/realm under 'Advanced Settings'. No need to change anything regarding the size of the read/write packages. These are negotiated between server and client automatically.
  
   <img src="images/DSM_Files_Services_NFS_Advanced_Settings.png" alt="Image: DSM Control Panel - File Sharing" width="400"/> 

* Import the `/etc/krbd5.keytab`file from the linux client. In this setup, it should only contain the keys for the service principal we are interested in, the NAS.

  <img src="images/DSM_Files_Services_NFS_Advanced_Settings_Kerberos_Keys.png" alt="Image: DSM Control Panel - File Sharing" width="400"/> 

* Map the principal ID from Kerberos database to a local user on the NAS.

  <img src="images/DSM_Files_Services_NFS_Advanced_Settings_ID-Mapping.png" alt="Image: DSM Control Panel - File Sharing" width="400"/>

  <img src="images/DSM_Files_Services_NFS_Advanced_Settings_ID-Mapping2.png" alt="Image: DSM Control Panel - File Sharing" width="400"/>

* Now the server's Kerberos principal identifier will show on the first screen of the 'Advanced Settings'.

 <img src="images/DSM_Files_Services_NFS_Advanced_Settings_Principal.png" alt="Image: DSM Control Panel - File Sharing" width="400"/>

* Create NFS permission rule for the file share you want to connect to with the Linux client.
  - Enter IP address of the Linux client
  - Read/write privilege
  - Do not enable any static mapping here
  - Security: choose either _Kerberos integrity_ (krb5i - Kerberos Authentication and Integrity Checking) or Kerberos Privacy (krb5p - Kerberos Authentication, Integrity Checking and Traffic Encryption). The first Kerberos option only does authentication and provides no further security. 

  <img src="images/DSM_Shared_Folder_NFS_Permissions.png" alt="Image: DSM Control Panel - File Sharing" width="400"/>
